In [1]:
%matplotlib notebook

import sys
import os
import time
from importlib import reload

In [2]:
import visa
visa.__version__

'1.9.0'

In [3]:
sys.path.append(r'd:/code/stationq/midas')

In [4]:
import numpy as np
import qcodes as qc
import broadbean as bb

from matplotlib import pyplot as plt
from broadbean.plotting import plotter

In [5]:
from qcodes.instrument.base import Instrument
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.plotting import plot_by_id
from qcodes.dataset.data_set import DataSet
from qcodes.sweep import sweep, nest, chain, szip, getter, setter
from qcodes.sweep.sweep import ParameterSweep, Nest, ParameterWrapper, FunctionWrapper, FunctionSweep, getter

In [6]:
from pytopo.qctools import instruments as instools; reload(instools)
from pytopo.qctools.instruments import create_inst, add2station

from pytopo.qctools import measurement; reload(measurement)
from pytopo.qctools.measurement import MeasurementExt

from pytopo.mplplots.init_nb_plotting import *
from pytopo.mplplots import plots as mplplots
from pytopo.mplplots import tools as mpltools

In [7]:
import plottr; reload(plottr)

from pytopo.qctools.dataset2 import select_experiment
from plottr.qcodes_dataset import QcodesDatasetSubscriber

In [8]:
from pytopo.qctools import hard_sweep; reload(hard_sweep)
from pytopo.qctools.hard_sweep import HardSweep, HardSweepDetector

In [9]:
inst_list = []


from qcodes.instrument_drivers.Minicircuits.RUDAT_13G_90 import RUDAT_13G_90_USB
ro_attenuator = instools.create_inst(RUDAT_13G_90_USB, 'ro_attenuator')
inst_list.append(ro_attenuator)

Connected to: Mini-Circuits MN=RUDAT-13G-90 (serial:SN=11704210052, firmware:C9-2) in 0.61s


In [10]:
from midas import Midas
midas = instools.create_inst(Midas, 'midas', '169.254.102.25', force_new_instance=True)


station = qc.Station(ro_attenuator, midas)

Connected to: Sydney Nanoscience Hub MIDAS (serial:0001, firmware:1.0) in 0.65s


In [11]:
frqs = np.linspace(100e6, 400e6, 301)
ro_attenuator.attenuation(0)

In [12]:
exp = select_experiment('MIDAS_test', 'RF_cond_test')

meas = Measurement(exp, station)
# meas.register_parameter(midas.ch1.frequency)
meas.register_custom_parameter('att')
meas.register_custom_parameter('frq')
# meas.register_custom_parameter('idx')
meas.register_custom_parameter('logmag', setpoints=['att', 'frq'])
# meas.register_parameter(midas.ch1.magnitude, setpoints=[midas.ch1.frequency, 'idx'])

meas.write_period = 1

with meas.run() as datasaver:
    datasaver.dataset.subscribe(
        QcodesDatasetSubscriber(datasaver.dataset), 
        state=[], min_wait=0, min_count=1,
    )
    
    for att in np.arange(0, 20, 2):
        ro_attenuator.attenuation(att)
        time.sleep(0.1)
        for f in frqs:
            midas.ch1.frequency(f)
            midas.soft_trigger_1d_trace()
            mag = midas.ch1.magnitude().mean()**2.
            mag = np.log10(mag)

            datasaver.add_result(
                ('att', att),
                ('frq', f), 
                ('logmag', mag))

Starting experimental run with id: 305
